In [1]:
# importing libraries
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import matplotlib.patches as mpatches
import colorcet as cc
mpl.rcParams['font.family'] = 'Times New Roman'

In [2]:
brits = [] # limits of noise level, tuned for each image

In [ ]:
imgs = []
bin_imgs = []
outers = []
names = []
i = 0
for file in sorted(os.listdir('data')):# looping through files in the folder
    if '.tif' in file: # checking image format
        img = cv2.imread(os.path.join('data', file), cv2.IMREAD_GRAYSCALE)
        bin_img = (img > brits[i]) * 255
        i += 1
        # using big kernel size to find most prominent edges
        sobelx = cv2.Sobel(bin_img.astype(np.float64), cv2.CV_64F, 1, 0, ksize=7)
        sobely = cv2.Sobel(bin_img.astype(np.float64), cv2.CV_64F, 0, 1, ksize=7)
        sobel_mag = cv2.magnitude(sobelx, sobely)
        sobel_mag_uint8 = cv2.convertScaleAbs(sobel_mag)
        _, binary_edges = cv2.threshold(sobel_mag_uint8, 50, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(binary_edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours_sorted = sorted(contours, key=cv2.contourArea, reverse=True)
        outer = contours_sorted[0]
        # performing morphological closing operations for certain boundaries that weere not fully formed
        if i in [10, 16]:
            sobelx = cv2.Sobel(img.astype(np.float64), cv2.CV_64F, 1, 0, ksize=7)
            sobely = cv2.Sobel(img.astype(np.float64), cv2.CV_64F, 0, 1, ksize=7)
            sobel_mag = cv2.magnitude(sobelx, sobely)
            sobel_mag_uint8 = cv2.convertScaleAbs(sobel_mag)
            _, binary_edges = cv2.threshold(sobel_mag_uint8, 50, 255, cv2.THRESH_BINARY)
            kernel = np.ones((5, 5), np.uint8)
            opened_edges = cv2.morphologyEx(binary_edges, cv2.MORPH_OPEN, kernel)
            closed_edges = cv2.morphologyEx(opened_edges, cv2.MORPH_CLOSE, kernel)
            contours, _ = cv2.findContours(closed_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours_sorted = sorted(contours, key=cv2.contourArea, reverse=True)
            outer = contours_sorted[0]
        imgs.append(img)
        bin_imgs.append(bin_img)
        outers.append(outer)
        names.append(file)

# plotting resutls in a grid
fig, axes = plt.subplots(nrows=18, ncols=1, figsize=(11, 5 * 18))
cmap = plt.get_cmap('cet_bgy')

for i in range(18):
    axes[i].imshow(imgs[i], cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(names[i].split('_')[0])

    edge = np.zeros_like(imgs[i])
    cv2.drawContours(edge, outers[i], -1, (255), thickness=5)
    axes[i].imshow(edge, cmap=cmap, alpha=0.5)

plt.tight_layout()
plt.show()